<table border="0" style="width:100%">
 <tr>
    <td>
        <img src="https://static-frm.ie.edu/university/wp-content/uploads/sites/6/2022/06/IE-University-logo.png" width=150>
     </td>
    <td><div style="font-family:'Courier New'">
            <div style="font-size:25px">
                <div style="text-align: right"> 
                    <b> MASTER IN BIG DATA</b>
                    <br>
                    Python for Data Analysis II
                    <br><br>
                    <em> Daniel Sierra Ramos </em>
                </div>
            </div>
        </div>
    </td>
 </tr>
</table>

In [ ]:
import json
import zipfile
import geopandas

import numpy as np
import pandas as pd

import plotly.express as px

## **Streamlit** App with Uber data

Build a Streamlit app to represent some charts about using public Uber data. This data contains information about the average travel time of Uber rides between all neighborhoods in MAdrid city.

The app must have the following characteristics
 - Main frame must show 3 charts given a `SOURCE` and `DESTINATION`:
    1. Time series of the average travel time
    2. Bar chart of the average travel time for every weekday, and period of the day
    3. **(Optional)** Choropleth map of Madrid city, with the source and destination highlighted
 - A side bar containing two box selectors to select the `SOURCE` and `DESTINATION`
 - Every time a source or a destination is selected, the charts of the main frame should be updated accordingly.

 NOTE: I recommend you to build the charts first here in the notebook, and then copy the code to the Streamlit app.

### Load data

Use the following function to load the data. For the first two figures you will need just the `data` variable. For the choropleth map you will need the `codes` variable and the `geopandas` library to work with maps (this is optional). 

In [6]:
def read_and_preprocess_data():
    
    with zipfile.ZipFile('data/uber-data.zip') as zip:
        with zip.open('madrid-barrios-2020-1-All-DatesByHourBucketsAggregate.csv') as csv:
            data = pd.read_csv(csv)
        with zip.open('madrid_barrios.json') as geojson:
            codes = geopandas.read_file(geojson, encoding="utf-8")

    # change data types in codes because they are not the same as in data
    codes['GEOCODIGO'] = codes['GEOCODIGO'].astype(int)
    codes['MOVEMENT_ID'] = codes['MOVEMENT_ID'].astype(int)

    codes["DISPLAY_NAME"] = codes["DISPLAY_NAME"].str.split().str[1:].str.join(" ")

    # Merge the data with the codes (source)
    data = data.merge(codes[["GEOCODIGO","MOVEMENT_ID","DISPLAY_NAME"]], left_on="sourceid", right_on="MOVEMENT_ID", how="left")
    data = data.rename(columns={"GEOCODIGO":"src_neigh_code", "DISPLAY_NAME":"src_neigh_name"}).drop(columns=["MOVEMENT_ID"])

    data = data.merge(codes[["GEOCODIGO","MOVEMENT_ID","DISPLAY_NAME"]], left_on="dstid", right_on="MOVEMENT_ID", how="left")
    data = data.rename(columns={"GEOCODIGO":"dst_neigh_code", "DISPLAY_NAME":"dst_neigh_name"}).drop(columns=["MOVEMENT_ID"])

    # Create a new date column
    data["year"] = "2020"
    data["date"] = pd.to_datetime(data["day"].astype(str)+data["month"].astype(str)+data["year"].astype(str)+":"+data["start_hour"].astype(str), format="%d%m%Y:%H")

    # Create a new day_period column
    data["day_period"] = data.start_hour.astype(str) + "-" + data.end_hour.astype(str)
    data["day_of_week"] = data.date.dt.weekday
    data["day_of_week_str"] = data.date.dt.day_name()

    return data, codes

### Travel by Time and Day

### Travel by Time and Day Period

### Map between source and destination